## Load libraries, Set Filepaths

In [1]:
import os
from utils import *
import keras.layers.convolutional as convolutional
from keras.models import Sequential
import allnew_vgg
reload(allnew_vgg)
import allnew_vgg as av
reload(allnew_vgg)

Using Theano backend.


<module 'allnew_vgg' from 'allnew_vgg.pyc'>

In [2]:
current_dir = os.getcwd()
data_dir = current_dir + '/data/kag_dogscats/' #sample/
# data_dir = current_dir + '/data/kag_dogscats/sample/'
results_path=data_dir + 'results/'
train_path=data_dir + 'train/'
valid_path=data_dir + 'valid/'
test_path=data_dir + 'test/'
model_path=data_dir + 'models/'

## Set variables

In [3]:
batch_size = 64
no_of_epochs = 8

## Build VGG model and load pretrained weights. Then Finetune

In [4]:
vgg = av.preloaded_vgg()

24576/35363 [===================>..........] - ETA: 0s

In [ ]:
vgg.mnn_finetuner()

In [ ]:
conv_layers, fc_layers = vgg.layer_divider()

## Take convolutional layers and build new model class to work off of

In [148]:
mgg = av.adjusted_vgg()

In [149]:
mgg.create(conv_layers)

## Gather Batches

In [150]:
train = mgg.mnn_batches(train_path, batch_size=batch_size, train=True, shuffle=False)
valid = mgg.mnn_batches(valid_path, batch_size=batch_size*2, train=False, shuffle=False)

train_classes = train.classes
valid_classes = valid.classes
train_labels = onehot(train_classes)
valid_labels = onehot(valid_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## Train weights for base model built above, save them to iterate new layers

In [40]:
train_features = mgg.model.predict_generator(generator=train, val_samples=train.nb_sample)
valid_features = mgg.model.predict_generator(generator=valid, val_samples=valid.nb_sample)

In [41]:
save_array(model_path+'train_convlayer_features.bc', train_features)
save_array(model_path+'valid_convlayer_features.bc', valid_features)

## Load base weights

In [132]:
# train_features = load_array(model_path+'train_convlayer_features.bc')
# valid_features = load_array(model_path+'valid_convlayer_features.bc')

## Create a model of just dense layers, using shape of trained features above

In [151]:
fc_model = mgg.dense_layer_model(fc_layers, input_shape=train_features.shape[1:])

In [51]:
fc_model.fit(x=train_features, y=train_labels, nb_epoch=8, batch_size=64, validation_data=(valid_features, valid_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/8
23000/23000 [==============================] - 33s - loss: 0.0633 - acc: 0.9780 - val_loss: 0.0316 - val_acc: 0.9865
Epoch 2/8
23000/23000 [==============================] - 34s - loss: 0.0188 - acc: 0.9944 - val_loss: 0.0323 - val_acc: 0.9880
Epoch 3/8
23000/23000 [==============================] - 34s - loss: 0.0073 - acc: 0.9986 - val_loss: 0.0795 - val_acc: 0.9855
Epoch 4/8
23000/23000 [==============================] - 34s - loss: 0.0032 - acc: 0.9997 - val_loss: 0.0848 - val_acc: 0.9870
Epoch 5/8
23000/23000 [==============================] - 34s - loss: 0.0028 - acc: 0.9998 - val_loss: 0.1193 - val_acc: 0.9845
Epoch 6/8
23000/23000 [==============================] - 34s - loss: 0.0028 - acc: 0.9998 - val_loss: 0.1005 - val_acc: 0.9880
Epoch 7/8
23000/23000 [==============================] - 33s - loss: 0.0028 - acc: 0.9998 - val_loss: 0.1052 - val_acc: 0.9865
Epoch 8/8
23000/23000 [==============================] - 33s -

In [152]:
fc_model.save_weights(results_path+'dogcats_clean.h5')
# fc_model.load_weights(results_path+'dogcats_clean.h5')

## Combine Weights from New Model with Old Model

In [159]:
for layer in fc_layers: 
    mgg.model.add(layer)

for l1,l2 in zip(fc_model.layers, mgg.model.layers[len(conv_layers):]):
    l2.set_weights(l1.get_weights())

## Generate Test Batches and Make Predictions

In [161]:
test_batches = get_batches(test_path, shuffle=False, batch_size=batch_size*2, class_mode=None)

Found 12500 images belonging to 1 classes.


In [163]:
predictions = mgg.model.predict_generator(test_batches, test_batches.nb_sample)

In [165]:
filenames = test_batches.filenames

save_array(results_path + 'test_predictions.dat', predictions)
save_array(results_path + 'filenames.dat', filenames)

## Formatting submission correctly, including "clipped" max and min

In [169]:
subm = mgg.submission_formatter(filenames, predictions, 1, .055, .945)

## Saving Predictions CSV and Generating a Click-able Link 

In [170]:
np.savetxt(results_path+'clean_model_submission.csv', subm, fmt='%d,%.5f', header='id,label', comments='')

In [186]:
# if filelink is not working, use commands below in command line
# %kg submit results/clean_model_submission.csv -m "Adding dropout and augmentation to VGG16."

ERROR:root:Line magic function `%kg` not found.


/home/ubuntu/courses/deeplearning1/nbs/results


In [183]:
# from IPython.display import FileLink
# FileLink(results_path+'clean_model_submission.csv')

/home/ubuntu/courses/deeplearning1/nbs/data/kag_dogscats/results/clean_model_submission.csv